# Image Classification Using Keras on MNIST Dataset


In [20]:
import tensorflow as tf

Loading and preparing the MNIST dataset.


In [21]:
mnist = tf.keras.datasets.mnist

In [22]:
type(mnist)

module

In [23]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Converting the samples from integers to floating-point numbers

In [24]:
x_train, x_test = x_train / 255.0, x_test / 255.0

# Model Building  using Keras
 building the tf.keras.Squential model by stacking layers. Choose an optimizer and los function for training

In [25]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Flatten(input_shape= (28,28)),
                                    tf.keras.layers.Dense(128,activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(10)
])


For each example the model returns a vector of 'logits' or 'log-odds' scores, one for each class

In [26]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.4924894 , -0.01833841, -0.16299438,  0.57751346,  0.48457155,
         0.1487886 , -0.41981184, -0.23201777,  0.17015243,  0.22317195]],
      dtype=float32)

the tf.nn.softmax function converts these ogits to 'prbablitites' for each class

In [27]:
tf.nn.softmax(predictions).numpy()

array([[0.13729277, 0.08237548, 0.07128115, 0.14947659, 0.13621001,
        0.09735995, 0.05513664, 0.06652705, 0.0994623 , 0.10487806]],
      dtype=float32)

In [28]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [29]:
loss_fn(y_train[:1], predictions).numpy()

2.3293402

In [30]:
model.compile(optimizer='adam',
              loss= loss_fn,
              metrics = ['accuracy'])

In [31]:
model.fit(x_train,y_train, epochs=50)

Epoch 1/50
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2938 - accuracy: 0.9139
Epoch 2/50
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1409 - accuracy: 0.9580
Epoch 3/50
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1081 - accuracy: 0.9674
Epoch 4/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0868 - accuracy: 0.9736
Epoch 5/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0749 - accuracy: 0.9768
Epoch 6/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0640 - accuracy: 0.9797
Epoch 7/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0598 - accuracy: 0.9809
Epoch 8/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0517 - accuracy: 0.9834
Epoch 9/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0482 - accuracy: 0.9840
Epoch 10/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.044

Model Evaluation On test set

In [32]:
model.evaluate(x_test,y_test, verbose= 2)

313/313 - 1s - loss: 0.1118 - accuracy: 0.9822


[0.11181827634572983, 0.982200026512146]

For Probablity, we are wraping the trained model and attaching the softmax to it

In [33]:
probablity_model = tf.keras.Sequential([
                                        model,
                                        tf.keras.layers.Softmax()
])

In [34]:
probablity_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.0752634e-16, 3.0745633e-22, 1.5143476e-12, 2.5119248e-06,
        7.0788891e-29, 5.9691410e-12, 2.7523533e-30, 9.9999750e-01,
        6.5373883e-12, 3.8725614e-10],
       [7.4297789e-13, 1.6735637e-08, 1.0000000e+00, 1.5650801e-15,
        0.0000000e+00, 3.0589600e-25, 3.5532989e-21, 0.0000000e+00,
        5.0994499e-20, 2.6190892e-36],
       [5.9861593e-18, 1.0000000e+00, 4.7200433e-13, 8.2858059e-15,
        3.0463371e-10, 2.3866136e-14, 8.3164592e-15, 3.2963898e-10,
        6.5713116e-11, 4.1967577e-20],
       [1.0000000e+00, 2.3637007e-17, 2.9875371e-09, 7.9637814e-18,
        2.9949228e-18, 6.2359737e-13, 5.0698907e-12, 3.9911959e-14,
        1.0955856e-16, 8.1336732e-14],
       [7.5843865e-19, 2.2107743e-32, 1.8579580e-14, 2.1870985e-21,
        9.9992490e-01, 2.5041049e-16, 1.8669415e-18, 3.6650391e-11,
        1.0637710e-19, 7.5153905e-05]], dtype=float32)>

# Saving Model

In [35]:
from keras.models import model_from_json

In [39]:
# serialize the model to JSON
model_json = model.to_json()
with open("model.json",'w') as json_file:
  json_file.write(model_json)

In [41]:
# Serialize weights to HDF5
model.save_weights("model.h5")
print("saved model to disk")

saved model to disk


Load Model

In [42]:
#load json and crate model
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()

In [44]:
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights('model.h5')


In [46]:
# evalute model on test data
loaded_model.compile(optimizer='adam', loss='SparseCategoricalCrossentropy',
                     )


In [48]:
loaded_model.fit(x_train,y_train, epochs= 20)

Epoch 1/20
1875/1875 [==============================] - 15s 2ms/step - loss: 0.4004
Epoch 2/20
1875/1875 [==============================] - 3s 2ms/step - loss: 1.3724
Epoch 3/20
1875/1875 [==============================] - 3s 2ms/step - loss: 1.6544
Epoch 4/20
1875/1875 [==============================] - 3s 2ms/step - loss: 1.7893
Epoch 5/20
1875/1875 [==============================] - 3s 2ms/step - loss: 1.9292
Epoch 6/20
1875/1875 [==============================] - 3s 2ms/step - loss: 1.9406
Epoch 7/20
1875/1875 [==============================] - 3s 2ms/step - loss: 1.9353
Epoch 8/20
1875/1875 [==============================] - 3s 2ms/step - loss: 1.7481
Epoch 9/20
1875/1875 [==============================] - 3s 2ms/step - loss: 1.7597
Epoch 10/20
1875/1875 [==============================] - 3s 2ms/step - loss: 1.9105
Epoch 11/20
1875/1875 [==============================] - 3s 2ms/step - loss: 1.8280
Epoch 12/20
1875/1875 [==============================] - 3s 2ms/step - loss: 1.8650
